In [19]:
'''
Created on 2016. 6. 17
@author: dato
@desc: cnlp hw12 (Midden Markov Model)
'''
#-*- coding: utf-8 -*-

import io
import numpy as np
import scipy as sp
from scipy import stats

In [20]:
def max_index(M):
    
    max = 0
    index = 0
    
    for i in range( len(M) ) :
        
        if M[i] > max: 
            max = M[i]
            index = i
        
    return index

# prob: 1 sample generation

In [22]:
def sample_gen(N, A, B, pi):
    
    y = [] # output
    
    z = [] # output prob.
    ll = []
    
    state_prob = pi   # 0='Fair' , 1='Bias'
    
    a_data = []
    b_data = []
    
    for i in range(A.shape[1]):
        a_data.append(i)
    
    for i in range(B.shape[1]):
        b_data.append(i)
    
    a_data = np.asarray(a_data)
    b_data = np.asarray(b_data)
    
    for i in range(N):
        # state prob = cur state X transition prob.
        state_prob = state_prob.dot(A)
        
        # select one state according to the state probability
        select = stats.rv_discrete(name='custm', values=(a_data, state_prob)).rvs(size=1)
#         select = max_index( state_prob )
        
        symbol_prob = B[select]
        # select one symbol according to the emission probability
        symbol = stats.rv_discrete(name='custm', values=(b_data, symbol_prob)).rvs(size=1)

        y.append(symbol[0])
        z.append(state_prob[select][0])
        ll.append( np.log(state_prob[select][0]) )

    return y, z, ll

In [23]:
A = np.array([[0.7, 0.3], [0.9, 0.1]])
B = np.array([[0.5, 0.5], [0.2, 0.8]])
pi= np.array([0.5,0.5])

y, z, l = sample_gen(10, A, B, pi)
print 'sequence \n' + str(y)
print 'probability \n' + str(z)
print 'log liklihood \n' + str(l)

ValueError: xk and pk need to have the same length.

# prob 2: forward backward

In [1124]:
def forward(Y, A, B, pi):    
    N = len(Y)

    a = np.ndarray(shape=(N,len(pi)), dtype=float)

    # initial value
    for k in range(len(pi)):
        a[0][k] = pi[k]*B[k][Y[0]]

    # loop
    # n : step, k : state (0, 1, 2, ...)
    for n in range(1, N):

        for k in range(len(pi)):

            #inner loop
            for j in range(len(pi)):
                a[n][k] += a[n-1][j] * A[j][k]

            a[n][k] *= B[k][Y[n]]

    eval = sum(a[len(a)-1])
            
    return a, eval

In [1125]:
def backward(Y, A, B, pi):   

    N = len(Y) -1
    eval = 0

    b = np.ndarray(shape=(N+1,len(pi)), dtype=float)

    # initial value
    for k in range(len(pi)):
        b[N][k] = 1

    # # loop
    # # n : step, k : state (0, 1, 2, ...)
    for n in reversed(range(N)):

        for k in range(len(pi)):

            #inner loop
            for j in range(len(pi)):
                b[n][k] += A[k][j] * B[j][Y[n+1]] * b[n+1][j]

    for k in range(len(pi)):
        eval += pi[k] * B[k][Y[0]] * b[0][k]
                
    return b, eval

In [1126]:
def forward_backward(Y, a, b, pi):
    
    eval = 0
    
    for k in range(len(pi)):
        eval += a[len(Y)-1][k] * b[ len(Y) -1][k]
        
    return eval

In [1127]:
A = np.array([[0.7, 0.3], [0.9, 0.1]])
B = np.array([[0.5, 0.5], [0.2, 0.8]])
pi= np.array([0.5,0.5])
Y = [0, 0, 1, 0, 0, 1, 0, 0, 0, 1]


a, eval = forward(Y, A, B, pi)
print '\n model evaluation with forward procedure: ' + str(eval)
print '\n' + str(a)


b, eval = backward(Y, A, B, pi)
print '\n model evaluation with backward procedure: ' + str(eval)
print '\n' + str(b)

eval = forward_backward(Y, a, b, pi)
print '\n model evaluation with forward/backward procedure: ' + str(eval)


 model evaluation with forward procedure: 1.58502375979

[[ 0.25        0.1       ]
 [ 0.1338594   0.01747332]
 [ 0.05690425  0.0374757 ]
 [ 0.04199931  0.00658977]
 [ 0.03064138  0.0071656 ]
 [ 0.03482664  0.04572085]
 [ 0.08274921  0.02555542]
 [ 0.15961216  0.06069606]
 [ 0.37817748  0.11679065]
 [ 0.68491791  0.90010585]]

 model evaluation with backward procedure: 0.000400472761349

[[ 0.00109357  0.00127079]
 [ 0.0027188   0.00236658]
 [ 0.00438093  0.00493948]
 [ 0.01043751  0.01212999]
 [ 0.02595245  0.02256924]
 [ 0.04175527  0.04724209]
 [ 0.099971    0.112757  ]
 [ 0.2383      0.2761    ]
 [ 0.59        0.53      ]
 [ 1.          1.        ]]

 model evaluation with forward/backward procedure: 1.58502375979


# prob 3: gamma (simple decoding)

In [1128]:
def decoding(Y, a, b):

    N = len(Y) -1
    eval = 0

    r = np.ndarray(shape=(N,len(pi)), dtype=float)

    for n in range (len(r)):

        for k in range(len(pi)):

            r[n][k] = a[n][k] * b[n][k]
        
        r[n] = r[n] / sum(r[n])
        

    return np.argmax(r, axis=1), r

In [1129]:
z, r = decoding(Y, a, b)

In [1130]:
print 'decoded states \n ' + str(z) + '\n'
print 'sequence probability (r_nk) \n' + str(r)

decoded states 
 [0 0 0 0 0 1 0 0 0]

sequence probability (r_nk) 
[[ 0.68267719  0.31732281]
 [ 0.89796924  0.10203076]
 [ 0.57387471  0.42612529]
 [ 0.84577762  0.15422238]
 [ 0.83100086  0.16899914]
 [ 0.40236238  0.59763762]
 [ 0.74165921  0.25834079]
 [ 0.69415893  0.30584107]
 [ 0.78282847  0.21717153]]


# Baum-Welch algorithm

generate true sequence & sampling

In [1113]:
# K = 2, R = 3

A = np.array([[0.8, 0.2], [0.4, 0.6]])
B = np.array([[0.2, 0.3, 0.5], [0.8, 0.1, 0.1]])
pi= np.array([0.3,0.7])
N = 100
Y, z, ll = sample_gen(N, A, B, pi)

In [1114]:
A_hat = np.array([[0.5, 0.5], [0.5, 0.5]])
B_hat = np.array([[0.3, 0.3, 0.4], [0.3, 0.3, 0.4]])
pi_hat = np.array([0.5,0.5])

zeta = np.ndarray(shape=(N,2,2), dtype=float)
gamma = np.ndarray(shape=(N,2), dtype=float)

In [1115]:
a, eval = forward(Y, A_hat, B_hat, pi_hat)
b, eval = backward(Y, A_hat, B_hat, pi_hat)

In [1116]:
for n in range(N-1):
    for j in range(2):
        for k in range(2):
            zeta[n][j][k] = a[n][j] * A_hat[j][k] * B_hat[k][Y[n+1]] * b[n+1][k]
        
    zeta[n] = zeta[n] / sum(zeta[n])
    
for n in range(N):
    for j in range(2):
        gamma[n][j] = a[n][j] * b[n][j]
    
    gamma[n] = gamma2[n] / sum(gamma2[n])